# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [6]:
!helm install ../helm-charts/seldon-core-operator --name seldon-core --set usageMetrics.enabled=true --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Fri Aug  9 16:53:58 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  0s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-config            1     0s
seldon-spartakus-config  1     0s

==> v1/Pod(related)
NAME                                        READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0        0/1    ContainerCreating  0         0s
seldon-spartakus-volunteer-df68b9d48-pg7s5  0/1    ContainerCreating  0         0s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     0s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.10

In [7]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

Waiting for 1 pods to be ready...
partitioned roll out complete: 1 new pods have been updated...


## Setup Ingress
There are gRPC issues with the latest Ambassador, so we rewcommend 0.40.2 until these are fixed.

In [8]:
!helm install stable/ambassador --name ambassador --set crds.keep=false

NAME:   ambassador
LAST DEPLOYED: Fri Aug  9 16:54:15 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        READY  UP-TO-DATE  AVAILABLE  AGE
ambassador  0/3    3           0          0s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-6d49d6b5c7-bvbm4  0/1    ContainerCreating  0         0s
ambassador-6d49d6b5c7-gzdlt  0/1    ContainerCreating  0         0s
ambassador-6d49d6b5c7-nc74z  0/1    ContainerCreating  0         0s

==> v1/Service
NAME               TYPE          CLUSTER-IP      EXTERNAL-IP  PORT(S)                     AGE
ambassador         LoadBalancer  10.106.248.204  <pending>    80:31591/TCP,443:31348/TCP  0s
ambassador-admins  ClusterIP     10.109.24.215   <none>       8877/TCP                    0s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        0s

==> v1beta1/ClusterRole
NAME        AGE
ambassador  0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
ambassador  0s

==>

In [9]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


### Port Forward to Ambassador

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Serve Single Model

In [10]:
!helm install ../helm-charts/seldon-single-model --name mymodel

NAME:   mymodel
LAST DEPLOYED: Fri Aug  9 17:02:59 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [10]:
!helm template ../helm-charts/seldon-single-model | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "RELEASE-NAME"
    },
    "spec": {
        "name": "RELEASE-NAME",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePe

In [11]:
!kubectl rollout status deploy/mymodel-mymodel-7cd068f

Waiting for deployment "mymodel-mymodel-7cd068f" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-7cd068f" successfully rolled out


### Get predictions

In [16]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [17]:
r = sc.predict(transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.8360570809602279
  }
}

Response:
meta {
  puid: "vtkhutqqekn2b71evuq40if5ds"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.11099580112955368
  }
}



#### gRPC Request

In [14]:
r = sc.predict(transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.11368719315764597
  }
}

Response:
meta {
  puid: "162o3utfp5rq903g0mm5r6ijf5"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.05716338296280312
  }
}



In [15]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [21]:
!helm install ../helm-charts/seldon-abtest --name myabtest

NAME:   myabtest
LAST DEPLOYED: Sat Jul 20 18:13:29 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME      AGE
myabtest  1s




In [19]:
!helm template ../helm-charts/seldon-abtest | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_1pod.json


---
# Source: seldon-abtest/templates/ab_test_2pods.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "RELEASE-NAME"
    },
    "spec": {
	"name": "RELEASE-NAME",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memo

In [22]:
!kubectl rollout status deploy/myabtest-abtest-41de5b8
!kubectl rollout status deploy/myabtest-abtest-df66c5c

deployment "myabtest-abtest-41de5b8" successfully rolled out
deployment "myabtest-abtest-df66c5c" successfully rolled out


### Get predictions

In [23]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="myabtest",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [30]:
r = sc.predict(transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.7647081632646227
  }
}

Response:
meta {
  puid: "8h4qnnqvo2861abjhsfdv5b267"
  routing {
    key: "myabtest"
    value: 0
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
    value: ""
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.10414835015130552
  }
}



#### gRPC Request

In [35]:
r = sc.predict(transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.3556469031281131
  }
}

Response:
meta {
  puid: "qbckh3cddrqlc78vvm27id9870"
  routing {
    key: "myabtest"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
    value: ""
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07168964710483954
  }
}



In [36]:
!helm delete myabtest --purge

release "myabtest" deleted


## Serve Multi-Armed Bandit

In [37]:
!helm install ../helm-charts/seldon-mab --name mymab

NAME:   mymab
LAST DEPLOYED: Sat Jul 20 18:15:30 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME   AGE
mymab  0s




In [26]:
!helm template ../helm-charts/seldon-mab | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "RELEASE-NAME"
    },
    "spec": {
	"name": "RELEASE-NAME",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }
			],
			"terminationGracePeriodSeco

In [38]:
!kubectl rollout status deploy/mymab-abtest-41de5b8
!kubectl rollout status deploy/mymab-abtest-b8038b2
!kubectl rollout status deploy/mymab-abtest-df66c5c 

deployment "mymab-abtest-41de5b8" successfully rolled out
deployment "mymab-abtest-b8038b2" successfully rolled out
deployment "mymab-abtest-df66c5c" successfully rolled out


### Get predictions

In [39]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymab",namespace="seldon",gateway_endpoint="localhost:8003",gateway="ambassador")

#### REST Request

In [40]:
r = sc.predict(transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.19835639554117446
  }
}

Response:
meta {
  puid: "15ur5egq097hkrj4kmot0lpsb2"
  routing {
    key: "eg-router"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.06190151573679043
  }
}



#### gRPC Request

In [42]:
r = sc.predict(transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.5350085315378481
  }
}

Response:
meta {
  puid: "u6fhq6blf6lrgesmse4tbudo2u"
  routing {
    key: "eg-router"
    value: 0
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08458209124227425
  }
}



In [43]:
!helm delete mymab --purge

release "mymab" deleted
